In [ ]:
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community
%pip install -Uq python_dotenv
%pip install PyPDF2
!pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 17.2 MB/s eta 0

In [2]:
import fitz  # PyMuPDF
import json
import os
from tqdm import tqdm
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

PDF_PATH = "dog-facts-the-pet-parent’s-atoz-home-care-encyclopedia.pdf"
CHROMA_PATH = "./chroma_db"

# === Step 1: Extract and split by page ===
print("📄 Extracting text and chunking per page...")
doc = fitz.open(PDF_PATH)

# Text splitter (customize chunk size and overlap as needed)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

all_chunks = []

for page_num in tqdm(range(len(doc)), desc="Chunking"):
    page = doc.load_page(page_num)
    text = page.get_text()
    if not text.strip():
        continue

    # Split page into smaller chunks
    chunks = text_splitter.split_text(text)

    # Attach page metadata
    for chunk in chunks:
        all_chunks.append(
            Document(
                page_content=chunk,
                metadata={"page": page_num + 1}
            )
        )

📄 Extracting text and chunking per page...


Chunking: 100%|██████████| 536/536 [00:01<00:00, 480.68it/s]


In [ ]:
len(all_chunks)

2998

In [ ]:
all_chunks[7]

Document(metadata={'page': 4}, page_content='I am grateful to all the cats and dogs I’ve met over the years who have shared my\nheart. Nine-year-old Magical-Dawg, 20-year-old Seren-Kitty, and the newcomer\nKarma-Kat inspire me daily.  Thanks be to ALL the dogs: those who live in our past,\nour present, and our future. I hope this book helps you with all of your furry wonders.\nI never would have been a pet lover, a reader and now a writer if not for my\nfantastic parents who instilled in me a love of the written word, and never looked')

In [3]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = Chroma(
    collection_name="rag_store",
    embedding_function=embeddings,
    persist_directory=CHROMA_PATH
)

vectorstore.add_documents(all_chunks)
vectorstore.persist()


/tmp/ipython-input-1935616323.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/tmp/ipython-input-193561

In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
query = "Why is my dog itching so much?"
results_with_scores = vectorstore.similarity_search_with_score(query, k=3)

for doc, score in results_with_scores:
    page = doc.metadata.get("page", "Unknown")
    print(f"📄 Page {page} | 🔍 Cosine Similarity Score: {score:.4f}")
    print(f"{doc.page_content[:200]}...\n")
    print("-" * 50)


📄 Page 309 | 🔍 Cosine Similarity Score: 0.5701
be associated with canine boredom. The affected dog incessantly licks a selected area,
usually on a lower leg, which creates a raised, hairless ulcerative plaque, almost a
callus that surrounds the ne...

--------------------------------------------------
📄 Page 309 | 🔍 Cosine Similarity Score: 0.5701
be associated with canine boredom. The affected dog incessantly licks a selected area,
usually on a lower leg, which creates a raised, hairless ulcerative plaque, almost a
callus that surrounds the ne...

--------------------------------------------------
📄 Page 42 | 🔍 Cosine Similarity Score: 0.6956
address these issues if your dog is also scratching a lot and paw-licking (see
ALLERGY).
The secretions become pasty and thick when not regularly expressed, and simply
plug the normal exit. Signs of a...

--------------------------------------------------


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
query = "Why is my dog eating grass?"
results_with_scores = vectorstore.similarity_search_with_score(query, k=3)

for doc, score in results_with_scores:
    page = doc.metadata.get("page", "Unknown")
    print(f"📄 Page {page} | 🔍 Cosine Similarity Score: {score:.4f}")
    print(f"{doc.page_content[:200]}...\n")
    print("-" * 50)


📄 Page 217 | 🔍 Cosine Similarity Score: 0.5542
roots, grasses and even fruit. Our dogs often beg for and enjoy snacks of raw
vegetables like lettuce, green beans and carrots.
Most pet dogs occasionally eat grass, which may be used as a natural eme...

--------------------------------------------------
📄 Page 217 | 🔍 Cosine Similarity Score: 0.5815
speculation exists that grass grazing may provide trace elements of vitamins.
 Indoor dogs may indulge their urge to graze by nibbling houseplants which,
depending on the plant, may be dangerous (see ...

--------------------------------------------------
📄 Page 126 | 🔍 Cosine Similarity Score: 0.6601
toilet area.
The behavior resolves in some dogs fed a high-protein low carbohydrate ration
twice a day. Adding a tablespoon of vegetable oil to the food every day seems to help
in some cases.
When all...

--------------------------------------------------


In [ ]:
import shutil

shutil.make_archive("chroma_db", 'zip', "chroma_db")


'/content/chroma_db.zip'

In [ ]:
%pip install -U langchain-community
%pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('open_router')

# === Setup your LLM ===
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENAI_API_KEY"],
    model="mistralai/mixtral-8x7b-instruct",
    temperature=0
)

# === Build the retriever wrapper ===
# 'query' is a string (the question)

# === Prompt builder (returns a ChatPromptValue) ===
def build_prompt(kwargs):
    docs = kwargs["context"]
    question = kwargs["question"]
    context = "\n\n".join(str(doc) for doc in docs)

    prompt = ChatPromptTemplate.from_template(
        """You are a professional and experienced veterinarian. Using only the information provided in the context below, answer the question as accurately and clearly as possible .

- Do not introduce external information.
- Do not make assumptions beyond what the context supports.
- Keep your answer concise and medically relevant.

Context:
{context}

Question:
{question}

Answer:"""
    )
    return prompt.format(context=context, question=question)

# === Compose the chain ===
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | RunnableLambda(build_prompt)
    | llm
    | StrOutputParser()
)

In [ ]:
query="How do I clean my dogs ears?"
response=chain.invoke(query)

print(response)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 According to the provided context, you should clean your dog's ears by gently wiping the visible portions of the ear with a 50/50 vinegar and water solution, or a commercial ear cleaning solution, using a cotton ball, soft cloth, or cotton swab. Do not drip cleaner into the ear canal or insert any instrument unless instructed by a veterinarian to avoid potential damage. If your dog has pendulous ears, like Cocker Spaniels, they may benefit from a weekly cleaning with a drying agent to prevent moisture buildup and potential ear infections.


In [ ]:
response = chain.invoke("my dog has rash on his stomach why is that")
print(response)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 Based on the information provided, there's no specific mention of a rash on the stomach. However, if the rash is characterized by red, crusty skin with pustules, it could potentially be a symptom of a bacterial infection related to a parasitic infestation, such as Demodicosis. This condition can cause skin issues and, in severe cases, can affect internal organs. If your dog is showing such symptoms, it would be best to consult with a veterinarian for a proper diagnosis and treatment.


In [ ]:
response = chain.invoke("How often should I bathe my dog?")
print(response)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 The frequency of bathing your dog depends on its coat type and lifestyle. For regularly groomed dogs with silky or wiry coats, baths four times a year are recommended. German Shepherds and similarly coated dogs should be bathed a couple of times a year, ideally in spring and fall. Smooth-coated pets like Rottweilers may need a bath only once a year. However, if your dog gets smelly or dingy-looking, or if it has encountered a skunk, it will require a bath sooner. Always ensure there are no mats in your dog's coat before bathing.


In [ ]:
eval_data = [
    {
        "question": "What is the average lifespan of a dog?",
        "ground_truth": "The average lifespan of a dog is 10 to 13 years, depending on the breed and health conditions."
    },
    {
        "question": "Are dogs omnivores or carnivores?",
        "ground_truth": "Dogs are omnivores; they can eat both meat and plant-based foods."
    },
    {
        "question": "Which breed is known as the fastest dog?",
        "ground_truth": "The Greyhound is known as the fastest dog breed, capable of reaching speeds up to 45 miles per hour."
    },
    {
        "question": "Can dogs see colors?",
        "ground_truth": "Dogs can see some colors, mainly blue and yellow, but they are not completely colorblind."
    },
    {
        "question": "Why do dogs wag their tails?",
        "ground_truth": "Dogs wag their tails to communicate emotions like happiness, excitement, or nervousness."
    },
    {
        "question": "What is the most intelligent dog breed?",
        "ground_truth": "The Border Collie is often considered the most intelligent dog breed."
    },
    {
        "question": "How many teeth does an adult dog have?",
        "ground_truth": "An adult dog typically has 42 teeth."
    },
    {
        "question": "Why do dogs pant?",
        "ground_truth": "Dogs pant to regulate their body temperature and cool down, especially after exercise or in hot weather."
    },
    {
        "question": "At what age do puppies start teething?",
        "ground_truth": "Puppies usually start teething at around 3 weeks old and continue until about 6 months."
    },
    {
        "question": "Can dogs eat chocolate?",
        "ground_truth": "No, chocolate is toxic to dogs and can cause serious health issues or even be fatal."
    }
]

import json

with open("dog_qa_eval_data1.json", "w", encoding="utf-8") as f:
    json.dump(eval_data, f, ensure_ascii=False, indent=2)

In [ ]:
!pip install nltk rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=500ddb6ed311b2529e645b5956a0b146e63e14202165dda4dd77ec3baf3080e4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re

# === Load data from JSON ===
with open("/content/dog_faqs_vet_answers.json", "r", encoding="utf-8") as f:
    eval_data = json.load(f)

# === Load embedding model ===
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# === Filtered lists ===
filtered_questions = []
filtered_ground_truths = []
predictions = []

# === Generate predictions and filter NA ===
for item in tqdm(eval_data, desc="Generating predictions"):
    q = item["question"]
    gt = item["answer"]

    answer = chain.invoke(q).strip()

    filtered_questions.append(q)
    filtered_ground_truths.append(gt)
    predictions.append(answer)

# === Encode both predictions and ground truths ===
pred_embs = model.encode(predictions, convert_to_tensor=True)
gt_embs = model.encode(filtered_ground_truths, convert_to_tensor=True)

# === Compute cosine similarity ===
cos_sim = util.cos_sim(pred_embs, gt_embs)
scores = cos_sim.diag().cpu().numpy()

# === Print results ===
print("\n🔍 Embedding Similarity per QA Pair:")
for i, score in enumerate(scores):
    print(f"Q{i+1}: {score:.4f}")
    print(f"Question: {filtered_questions[i]}")
    print(f"Prediction: {predictions[i]}")
    print(f"Ground Truth: {filtered_ground_truths[i]}")
    print()

print(f"\n✅ Average Embedding Similarity: {np.mean(scores):.4f}")

Generating predictions:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Generating predictions:  10%|█         | 1/10 [00:01<00:14,  1.61s/it]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Generating predictions:  20%|██        | 2/10 [00:03<00:12,  1.56s/it]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Generating predictions:  30%|███       | 3/10 [00:04<00:10,  1.49s/it]/usr/


🔍 Embedding Similarity per QA Pair:
Q1: 0.6727
Question: How often should I take my dog to the vet?
Prediction: The context suggests that for "core" vaccinations, either annual or every three years revaccination is typically recommended. Given that your dog will need to see the vet regularly for well-dog exams anyway, these visits can serve as opportunities to administer or discuss the core vaccines. Therefore, you should take your dog to the vet at least annually, but your vet may recommend specific intervals for different vaccines.
Ground Truth: At minimum, adult dogs should have an annual wellness exam. Puppies require more frequent visits for vaccinations and checkups, while senior dogs (7+ years) benefit from twice-yearly visits to catch age-related issues early.

Q2: 0.6440
Question: What vaccines does my dog need?
Prediction: Based on the provided context, the vaccines your dog needs depend on its risk factors, which are influenced by its age, immune competence, exposure, and s

#Agentic RAG

In [23]:
import os
from langchain.vectorstores.base import VectorStore
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai import Agent, Task, Crew


os.environ["OPENAI_API_KEY"] = userdata.get('open_router')

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENAI_API_KEY"],
    model="mistralai/mixtral-8x7b-instruct",
    temperature=0
)

def build_prompt(kwargs):
    docs = kwargs['context']
    question = kwargs['question']
    context = "\n\n".join(str(doc) for doc in docs)

    prompt = ChatPromptTemplate.from_template(
        """You are a professional and experienced veterinarian. Using only the information provided in the context below, answer the question as accurately and clearly as possible.

- Do not introduce external information.
- Do not make assumptions beyond what the context supports.
- Keep your answer concise and medically relevant.

Context:
{context}

Question:
{question}

Answer:"""
    )
    return prompt.format(context=context, question=question)

# --- RAG tool ---
def run_rag_chain(question):
    chain = (
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
        | RunnableLambda(build_prompt)
        | llm
        | StrOutputParser()
    )
    return chain.invoke(question)



web search

In [24]:
os.environ["TAVILY_API_KEY"] = userdata.get('tavily')

tavily_search = TavilySearchResults(api_key=os.environ["TAVILY_API_KEY"])

def run_web_search(question):
    results = tavily_search.run(question)

    return f"Web search results:\n{results}"

/tmp/ipython-input-1594566303.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(api_key=os.environ["TAVILY_API_KEY"])


In [26]:
from langchain.tools import Tool

rag_tool = Tool(
    name="InternalRAG",
    func=run_rag_chain,
    description="Use this tool to answer veterinary-medical questions when provided background context is sufficient."
)
web_tool = Tool(
    name="WebSearch",
    func=run_web_search,
    description="Use this tool for recent, novel, or general questions where the internal context doesn't suffice."
)

tools = [rag_tool, web_tool]

In [27]:
system_message = """
You are a veterinary assistant agent. Your task is to pick the best approach for each question:
- If the question can be answered from the internal veterinary knowledge base, use 'InternalRAG'.
- If the question is about recent developments, news, or if the internal context seems insufficient, use 'WebSearch' to search online.
Think step by step, then act.
"""

In [29]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={'system_message': system_message}
)

/tmp/ipython-input-1746318073.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [32]:
question = input("Ask your veterinary question: ")
answer = agent.run(question)
print("\nAnswer:\n", answer)

Ask your veterinary question: Why is my dog itching so much?


> Entering new AgentExecutor chain...
 The dog's excessive itching could be due to various reasons such as parasites, allergies, or skin conditions. I need more context about the dog's medical history to provide a precise answer.

Action: InternalRAG
Action Input: Why is my dog itching so much? The dog is a 5-year-old mixed breed with no known allergies or medical conditions.

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Observation:  Based on the provided context, intense all-over itchiness in dogs can be a symptom of a food allergy, especially in dogs that are two years old or older. However, your dog is a 5-year-old mixed breed with no known allergies or medical conditions. Therefore, the itchiness could be due to a variety of factors and not necessarily a food allergy. It would be best to consult with your veterinarian to identify the cause and appropriate treatment.
Thought: I now know that the dog's excessive itching could be due to various reasons such as parasites, allergies, or skin conditions, and it is necessary to consult with a veterinarian for a proper diagnosis and treatment.
Final Answer: Your dog's excessive itching could be due to various reasons such as parasites, allergies, or skin conditions. It would be best to consult with your veterinarian to identify the cause and appropriate treatment.

> Finished chain.

Answer:
 Your dog's excessive itching could be due to various reasons s

In [38]:
question = input("Ask your veterinary question: ")
answer = agent.run(question)
print("\nAnswer:\n", answer)


Ask your veterinary question:  Can dogs eat human food?


> Entering new AgentExecutor chain...
 This is a general question about dogs' diet, which the internal context might not have sufficient information. I should use the web search tool to find a more accurate answer.

Action: WebSearch
Action Input: "Can dogs eat human food?"
Observation: Web search results:
[{'title': 'Human Foods Dogs Can Eat: Complete List', 'url': 'https://thevets.com/resources/pet-nutrition/human-food-dogs-can-eat/', 'content': '## Frequently Asked Questions\n\n### Can dogs eat human food?\n\nYes, they can but only in moderation. Stick to safe human foods such as lean turkey and chicken (with no bones), carrots, broccoli, and blueberries.\n\n### What vegetables can dogs eat?\n\nDogs can eat carrots, broccoli, green beans, sweet potato, celery, and beets.\n\n### What is the best human food for dogs?\n\nVegetables such as carrots and broccoli are safe to eat along with boiled chicken and white rice which can he